In [12]:
import xml.etree.ElementTree as ET
import html2markdown
import os
from pymongo import MongoClient
import pprint as pp
import json

data_path = fr'{os.getenv("HOME")}/GitHub/RNCP34757BC02/brief-AGILE-chatBot/data'
# subdir = "workplace.meta.stackexchange.com"
file_name = "Posts.xml"

In [13]:
def add_posts(data, language, topic, Id=1, old_Id=1):
    posts = []
    for i in range(len(data['conversations'])):
        for j in range(2):
            if j == 0:
                PostTypeId = 1
                post = {
                    'Id': Id,
                    'PostTypeId': str(PostTypeId),
                    'Score': '1',
                    "ViewCount": "1",
                    'Title': data['conversations'][i][j],
                    "AnswerCount": "1",
                    'Language': language,
                    'Topic': topic,
                }
            else:
                PostTypeId = 2
                post = {
                    'Id': Id,
                    'PostTypeId': str(PostTypeId),
                    'ParentId': str(old_Id),
                    'Score': '1',
                    'Body': data['conversations'][i][j],
                    'Language': language,
                    'Topic': topic,
                }
            posts.append(post)
            old_Id = Id
            Id += 1
    return posts, Id, old_Id

In [15]:
for subdir in os.listdir(data_path):
    print(subdir)
    current_path = os.path.join(data_path, subdir)
    print(current_path)
    for file in os.listdir(current_path):
        if file == file_name:
            posts_tree = ET.parse(os.path.join(current_path, file))
            posts = posts_tree.getroot()

            if "meta" in subdir:
                topic = subdir[:-23]
            else:
                topic = subdir[:-17]

            posts_list = [row.attrib for row in posts]

            for post in post_list:
                post['Body'] = html2markdown.convert(post['Body'])
                post['Topic'] = topic            

JSON
/home/olivier/GitHub/RNCP34757BC02/brief-AGILE-chatBot/data/JSON


In [3]:
# Connexion to mongodb with pymongo
# Connection by default to db at localhost:27017
connection = MongoClient()

# STARKBOT DB creation
db = connection["StarkBotBD"]
collection = db.Quest_Rep
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'StarkBotBD'), 'Quest_Rep')

In [4]:
initial_present_docs = collection.count_documents({})
print(initial_present_docs, "documents in", db.name)

5774 documents in StarkBotBD


In [5]:
post = posts_list[0]
pp.pprint(post)

{'AcceptedAnswerId': '18',
 'AnswerCount': '1',
 'Body': 'I have been working on a "bot" for operating on various websites. I '
         'want the bot to be able to accept and send XLM between two parties, '
         "even if the receiving party doesn't yet have a XLM address. \n"
         '\n'
         'The idea is to build a way to ask questions to the community and get '
         'rewarded for the right answers. If people want, they can put a '
         '"bounty" on the question being asked, and then promise that the '
         '"bounty" is paid out to the best answer.\n'
         '\n'
         'Beyond creating an interim XLM address (which seems risky',
 'CommentCount': '1',
 'ContentLicense': 'CC BY-SA 3.0',
 'CreationDate': '2018-01-17T19:36:36.297',
 'FavoriteCount': '1',
 'Id': '1',
 'LastActivityDate': '2018-01-17T22:03:13.630',
 'LastEditDate': '2018-01-17T22:03:13.630',
 'LastEditorUserId': '33',
 'OwnerUserId': '34',
 'PostTypeId': '1',
 'Score': '-2',
 'Tags': '<xlm><trans

In [7]:
post["Topic"]

'stellar.'

In [11]:
for post in posts_list:
    query = collection.find({"$and": [{"Id": post["Id"], "Topic": post["Topic"]}]})
    if query.count() == 0:
        collection.insert_one(post)
        print("document inserted", post["_id"])

AttributeError: 'Cursor' object has no attribute 'count_documents'

In [ ]:
present_docs = db.Quest_Rep.count_documents({})
inserted_docs = present_docs - initial_present_docs
print(inserted_docs, "documents have been inserted in", db.name)
print(present_docs, "documents in", db.name)